# Stanford Dogs CNN 

In [32]:
# basic modeling
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np

# Loss Function
import torch.nn.functional as F
import torch.optim as optim

# Data Cleaning
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.transforms import ToTensor

In [7]:
# Directory where data is
data_dir = r'C:\Users\dsk02\Desktop\python_projects\torch_dogs\main_images\Images'
print(os.listdir(data_dir))

['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog', 'n02086079-Pekinese', 'n02086240-Shih-Tzu', 'n02086646-Blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-Rhodesian_ridgeback', 'n02088094-Afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-Walker_hound', 'n02089973-English_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-Irish_wolfhound', 'n02091032-Italian_greyhound', 'n02091134-whippet', 'n02091244-Ibizan_hound', 'n02091467-Norwegian_elkhound', 'n02091635-otterhound', 'n02091831-Saluki', 'n02092002-Scottish_deerhound', 'n02092339-Weimaraner', 'n02093256-Staffordshire_bullterrier', 'n02093428-American_Staffordshire_terrier', 'n02093647-Bedlington_terrier', 'n02093754-Border_terrier', 'n02093859-Kerry_blue_terrier', 'n02093991-Irish_terrier', 'n02094114-Norfolk_terrier', 'n02094258-Norwich_terrier', 'n02094433-Yorkshire_t

In [8]:
# clean up classes name
temp = os.listdir(data_dir)

classes = []

# class cleaning

for each_folder in temp:
    # split words on -
    split_words = each_folder.split('-')
    
    # if there is more than one dash
    if len(split_words) > 2:
        
        # create temp word
        temp = ''
        # for each value in the split_words array (starting from 1)
        for i in range(1,len(split_words)):
            
            # if it's not the end add word + space
            if i != (len(split_words)-1):
                temp += (split_words[i] + ' ')
            else:
                # if end just add the word
                temp += (split_words[i])
        
        # append temp to classes
        classes.append(temp.lower())
    
    # if it equals 2 it's just nasty title + name, append name
    elif len(split_words) == 2:
        classes.append(each_folder.split('-')[1].lower())
    
    # outliers append 
    else:
        classes.append(each_folder.lower())
    
# print first few classes, all lower case
print(classes)

['chihuahua', 'japanese_spaniel', 'maltese_dog', 'pekinese', 'shih tzu', 'blenheim_spaniel', 'papillon', 'toy_terrier', 'rhodesian_ridgeback', 'afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'black and tan_coonhound', 'walker_hound', 'english_foxhound', 'redbone', 'borzoi', 'irish_wolfhound', 'italian_greyhound', 'whippet', 'ibizan_hound', 'norwegian_elkhound', 'otterhound', 'saluki', 'scottish_deerhound', 'weimaraner', 'staffordshire_bullterrier', 'american_staffordshire_terrier', 'bedlington_terrier', 'border_terrier', 'kerry_blue_terrier', 'irish_terrier', 'norfolk_terrier', 'norwich_terrier', 'yorkshire_terrier', 'wire haired_fox_terrier', 'lakeland_terrier', 'sealyham_terrier', 'airedale', 'cairn', 'australian_terrier', 'dandie_dinmont', 'boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'scotch_terrier', 'tibetan_terrier', 'silky_terrier', 'soft coated_wheaten_terrier', 'west_highland_white_terrier', 'lhasa', 'flat coated_retriever', 'cur

In [9]:
# main dataset
# transform to tensors
# labels are folders name (yay torch)
dataset = ImageFolder(data_dir, transform=ToTensor())

In [27]:
# Length of data
print(len(dataset))

# Split data!!!
size = len(dataset)
train_size = int(size * .80)
val_size = int(size * .20)


# split into train & validation
train_ds, val_ds = random_split(dataset, [train_size, val_size])


size == (train_size + val_size)

20580


True

In [46]:
# Create Train & Test Loaders from dataloader

In [28]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2)

In [55]:
list(train_ds[0][0].size()) # need to resize these!!!!!!!

[3, 333, 500]

# Initialize Model

In [29]:
import torch.nn as nn
import torch.nn.functional as F

class StanfordDogs(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [30]:
model = StanfordDogs()

In [35]:
# Loss function & Optimizer

epochs = 8
max_lr = 0.01
# grad_clip = 0.1
# weight_decay = 1e-4

# cross entropy for classification
crit = nn.CrossEntropyLoss()

# george hotz says adam is the best
opt = optim.Adam(model.parameters(), lr=max_lr)

# Trainning Day

In [47]:
loss_arr = []

for i, data in enumerate(train_ds,0):
    # i in index, enumerate allows for tensor pass.. ignore and use
    
    # split data with inputs and labels
    inputs, labels = data
    
    # zero the parameter gradients
    opt.zero_grad()
    
    #forward + backward + optimize 
    
    # get predictions
    outputs = model(inputs)
    
    # calculate how bad they are
    loss = crit(outputs, labels)
    
    # calculate the loss wrt to all the parameters in loss (dx)
    loss.backward()
    
    #update your parameters
    opt.step()
    
    # loss increase
    ls += loss.item()
    loss_arr.append(loss.item())
    

for i,loss in enumerate(loss_arr):
    print('Iteration {} & Loss {%d2}'.format(i,loss))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 3, 3, 3], but got 3-dimensional input of size [3, 333, 500] instead